In [ ]:
import numpy as np
import re
import math
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
import pandas as pd
from PIL import Image
import json
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from object_detection.utils import visualization_utils
from object_detection.builders import model_builder
import xml.etree.ElementTree as ET
%matplotlib inline

Download tensorflow 2 object detection api
https://github.com/tensorflow/models/tree/master/research/object_detection

# Training 

In [ ]:
set XLA_FLAGS=--xla_gpu_cuda_data_dir=C:\Users\Imtiaz\anaconda3\envs\tf_gpu\Lib

In [ ]:
cd C:\Users\Imtiaz\Downloads\DL_models\models\research

In [ ]:
set PIPELINE_CONFIG_PATH=C:\\Users\\Imtiaz\\Downloads\\DL_models\\faster_rcnn_data\\pre_trained_model_resnet50_640_by_640\\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config
set MODEL_DIR=C:\\Users\\Imtiaz\\Downloads\\DL_models\\faster_rcnn_data\\train_events\\resnet50_evnt1

python object_detection/model_main_tf2.py ^
    --pipeline_config_path=%PIPELINE_CONFIG_PATH% ^
    --model_dir=%MODEL_DIR% ^
    --alsologtostderr


# Evaluation

In [ ]:
set PIPELINE_CONFIG_PATH=C:\\Users\\Imtiaz\\Downloads\\DL_models\\faster_rcnn_data\\pre_trained_model_resnet50_640_by_640\\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config
set MODEL_DIR=C:\\Users\\Imtiaz\\Downloads\\DL_models\\faster_rcnn_data\\train_events\\resnet50_evnt1
set CHECKPOINT_DIR=C:\\Users\\Imtiaz\\Downloads\\DL_models\\faster_rcnn_data\\train_events\\resnet50_evnt1


python object_detection/model_main_tf2.py ^
    --pipeline_config_path=%PIPELINE_CONFIG_PATH% ^
    --model_dir=%MODEL_DIR% ^
    --checkpoint_dir=%CHECKPOINT_DIR% ^
    --alsologtostderr

# Loading Test images

In [ ]:
test_image_paths = []
for root, dirs, files in os.walk('data\\images\\test'):
    for i in files:
        test_image_paths.append(
            os.path.join(os.getcwd(),root,i)
        )
test_image_paths

# Loading Model from checkpoint

In [ ]:
%cd C:\Users\Imtiaz\Downloads\DL_models\models\research

In [ ]:
config_path = "C:\\Users\\Imtiaz\\Downloads\\DL_models\\saved_models\\en\\pipeline.config"
config = config_util.get_configs_from_pipeline_file(config_path)

In [ ]:
detection_model = model_builder.build(model_config=config['model'], is_training=False)
# Restore checkpoint
checkpoint_path = "C:\\Users\\Imtiaz\\Downloads\\DL_models\\faster_rcnn_data\\train_events\\evn8"
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpoint_path, 'ckpt-3')).expect_partial()

In [ ]:
label_map_path_ = "C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\labelmap.pbtxt"
class_labels = label_map_util.create_category_index_from_labelmap(label_map_path_)

In [ ]:
@tf.function
def predict(input_image):
    img = cv2.imread(input_image)
    input_tensor = tf.convert_to_tensor(
        np.expand_dims(img, 0), dtype=tf.float32)
    image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

def detect(input_image):
    @tf.function
    def process(img):
        image, shapes = detection_model.preprocess(img)
        prediction_dict = detection_model.predict(image, shapes)
        detections = detection_model.postprocess(prediction_dict, shapes)
        return detections

    img = cv2.imread(input_image)
    input_tensor = tf.convert_to_tensor(
        np.expand_dims(img, 0), dtype=tf.float32)

    detections = process(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(
        np.int64)
    label_id_offset = 1
    image_with_detections = img.copy()
    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        class_labels,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=.8,
        agnostic_mode=False)

    return image_with_detections

In [ ]:
for i in range(len(test_image_paths)):
    detection_image = detect(test_image_paths[i])
    cv2.imwrite(f"pre_img\\pred1\\pre{i+1}.jpg", detection_image)

# Load from saved model

In [ ]:
model_path = "C:\\Users\\Imtiaz\\Downloads\\DL_models\\saved_models\\newFRCNNRESNET101EXP4\\saved_model"
loaded_model =  tf.saved_model.load(model_path)

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap("C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\labelmap.pbtxt",
                                                                    use_display_name=True)

In [ ]:
image_paths = []
for root, dirs, files in os.walk(os.path.join(os.getcwd(),'NE')):
    for i in files:
        image_paths.append(
            os.path.join(os.getcwd(),root,i)
        )
image_paths

In [ ]:

from utils.CE_modified import ContastEnhancement

def inference(path):
    try:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        # ce = ContastEnhancement(img)
        # Pce = ce.prcoess_image()
        image_np = np.array(img)
        input_tensor = tf.convert_to_tensor(image_np)
        input_tensor = input_tensor[tf.newaxis, ...]
        detections = loaded_model(input_tensor)
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        image_np_with_detections = image_np.copy()

        # Check if class name is 'other' and change it to 'light pole'
        if category_index[detections['detection_classes'][0]]['name'] == 'other':
            category_index[detections['detection_classes'][0]]['name'] = 'light pole'

        # Visualize detections
        visualization_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'],
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=0.8,
            agnostic_mode=False
        )

        return image_np_with_detections
    except Exception as e:
        print(e)


In [ ]:
import time
import re
# ... other imports

start_time = time.time()  # Record the start time before the loop

for i in range(len(image_paths)):
    name = re.findall(re.compile(r"[a-zA-Z-0-9\_]+"), image_paths[i])[-2]
    di = inference(image_paths[i])
    cv2.imwrite(f"results\\faster rcnn101NE\\{name}.png", di)

end_time = time.time()  # Record the end time after the loop
total_execution_time = end_time - start_time  # Calculate the total execution time

print(f"Total execution time for processing {len(image_paths)} images: {total_execution_time:.4f} seconds")

# Model Evaluation

Training localization and classification loss

In [ ]:
train_box_loss_cv = os.path.join(os.getcwd(
), "saved_models\\faster_rcnn_resnet101\\en\\train_loc_loss.csv")
train_box_loss_cv = pd.read_csv(train_box_loss_cv)
train_class_loss_cv = os.path.join(os.getcwd(
), "saved_models\\faster_rcnn_resnet101\\en\\train_class_loss.csv")
train_class_loss_cv = pd.read_csv(train_class_loss_cv)
train_total_loss = os.path.join(os.getcwd(
), "saved_models\\faster_rcnn_resnet101\\en\\train_total_loss.csv")
train_total_loss = pd.read_csv(train_total_loss)

In [ ]:
sns.lineplot(x="Step", y="Value", data=train_box_loss_cv)
plt.title("Training localization loss")
plt.show()

In [ ]:
sns.lineplot(x="Step", y="Value", data=train_class_loss_cv)
plt.title("Training classification loss")
plt.show()

In [ ]:
sns.lineplot(x="Step", y="Value", data=train_total_loss)
plt.title("Training Total loss")
plt.show()

Validation localization and classification loss

In [ ]:
val_box_loss_cv = os.path.join(os.getcwd(
), "saved_models\\faster_rcnn_resnet101\\en\\val_loc_loss.csv")
val_box_loss_cv = pd.read_csv(val_box_loss_cv)
val_class_loss_cv = os.path.join(os.getcwd(
), "saved_models\\faster_rcnn_resnet101\\en\\val_class_loss.csv")
val_class_loss_cv = pd.read_csv(val_class_loss_cv)
val_total_loss =  os.path.join(os.getcwd(
), "saved_models\\faster_rcnn_resnet101\\en\\val_total_loss.csv")
val_total_loss = pd.read_csv(val_total_loss)

In [ ]:
sns.lineplot(x="Step", y="Value", data=val_box_loss_cv)
plt.title("Validation localization loss")
plt.show()

In [ ]:
sns.lineplot(x="Step", y="Value", data=val_class_loss_cv)
plt.title("Validation classification loss")
plt.show()

In [ ]:
sns.lineplot(x="Step", y="Value", data=val_total_loss)
plt.title("Validation Total loss")
plt.show()

Confusion matrix, precision , recall and F1 score

In [ ]:
labels = {
    "tm": 1,
    "tnb": 2,
    "other": 3
}
filename = os.path.join(os.getcwd(), "annotations", "labels_dict.json")
with open(filename, "w") as json_file:
    json.dump(labels, json_file)
print("Labels have been written to", filename)

In [ ]:
import time

def save_to_csv(data):
    counter = 0
    image_pred = pd.DataFrame()
    inference_time = {"image":[], "time":[]}
    try:
        for image in data:
            print('Running inference for {}... '.format(image.split('/')[-1]), end='')
            start_time = time.time()  # Record start time
            img = cv2.imread(image, cv2.IMREAD_UNCHANGED)
            image_np = np.array(img)
            height, width, l = image_np.shape
            input_tensor = tf.convert_to_tensor(image_np)
            input_tensor = input_tensor[tf.newaxis, ...]
            detections = loaded_model(input_tensor)
            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                          for key, value in detections.items()}
            
            for i in range(num_detections):
                ll = []
                ll.append(image)
                ll.append(width)
                ll.append(height)
                ll.append(detections['detection_boxes'][i].tolist())
                ll.append(category_index[detections['detection_classes'][i]]['name'])
                ll.append(detections['detection_scores'][i])
            
                pre_df = pd.DataFrame([{'filename': ll[0],
                                        'width': ll[1],
                                        'height': ll[2],
                                        'class': ll[4],
                                        'xmin': ll[3][1] * width,
                                        'ymin': ll[3][0] * height,
                                        'xmax': ll[3][3] * width,
                                        'ymax': ll[3][2] * height,
                                        'score': ll[5]}])
                image_pred = pd.concat([image_pred, pre_df], axis=0)
            
            end_time = time.time()  # Record end time
            inference_time['image'].append(image)
            inference_time['time'].append(end_time - start_time)  # Calculate inference time
            counter += 1
            print(f'Done : {counter}')
        return image_pred, inference_time
    except Exception as e:
        print("image has issue :" + image.split('/')[-1])

In [ ]:
pred_data, pred_time = save_to_csv(image_paths)

In [ ]:
pred_data_path = os.path.join(
    os.getcwd(), "ssd_data\\train_events\\evn1", "pred_data.csv")
pred_time_path = os.path.join(
    os.getcwd(), "ssd_data\\train_events\\evn1", "pred_time.csv")

In [ ]:
pred_data.to_csv(pred_data_path, index=False)
pd.DataFrame(pred_time).to_csv(pred_time_path, index=False)

In [ ]:
ground_truth_csv = pd.read_csv(
    "dataset_csv\\test_data.csv"
)
pred_data_read = pd.read_csv(pred_data_path)
pred_time_read = pd.read_csv(pred_time_path)

In [ ]:
def compute_iou(groundtruth_box, detection_box):
    g_xmin, g_ymin, g_xmax, g_ymax = tuple(groundtruth_box)
    d_xmin, d_ymin, d_xmax, d_ymax = tuple(detection_box)
    xa = max(g_xmin, d_xmin)
    ya = max(g_ymin, d_ymin)
    xb = min(g_xmax, d_xmax)
    yb = min(g_ymax, d_ymax)
    intersection = max(0, xb - xa + 1) * max(0, yb - ya + 1)
    boxAArea = (g_xmax - g_xmin + 1) * (g_ymax - g_ymin + 1)
    boxBArea = (d_xmax - d_xmin + 1) * (d_ymax - d_ymin + 1)
    return intersection / float(boxAArea + boxBArea - intersection)


def process_detections(ground_truth_csv, pred_csv, IOU_THRESHOLD, CONFIDENCE_THRESHOLD, categories):
    confusion_matrix = np.zeros(
        shape=(len(categories) + 1, len(categories) + 1))
    file_unique = ground_truth_csv['filename'].unique()
    for file in file_unique:
        test_df = ground_truth_csv[ground_truth_csv['filename'] == file]
        test_df.reset_index(inplace=True, drop=True)
        pred_df = pred_csv[pred_csv['filename'] == file]
        pred_df.reset_index(inplace=True, drop=True)

        pred_class = pred_df[pred_df['score'] >= CONFIDENCE_THRESHOLD]

        groundtruth_boxes = test_df[[
            'xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
        detection_boxes = pred_class[[
            'xmin', 'ymin', 'xmax', 'ymax']].values.tolist()
        #print(groundtruth_boxes)
        #print(detection_boxes)
        matches = []

        for i in range(len(groundtruth_boxes)):
            for j in range(len(detection_boxes)):
                iou = compute_iou(groundtruth_boxes[i], detection_boxes[j])

                if iou > IOU_THRESHOLD:
                    matches.append([i, j, iou])

        matches = np.array(matches)

        #print(matches)

        if matches.shape[0] > 0:
            # Sort list of matches by descending IOU so we can remove duplicate detections
            # while keeping the highest IOU entry.
            matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]

            # Remove duplicate detections from the list.
            matches = matches[np.unique(matches[:, 1], return_index=True)[1]]

            # Sort the list again by descending IOU. Removing duplicates doesn't preserve
            # our previous sort.
            matches = matches[matches[:, 2].argsort()[::-1][:len(matches)]]

            # Remove duplicate ground truths from the list.
            matches = matches[np.unique(matches[:, 0], return_index=True)[1]]

        for i in range(len(groundtruth_boxes)):
            #print(i)
            if matches.shape[0] > 0 and matches[matches[:, 0] == i].shape[0] == 1:
                #print("inside : ", i)
                confusion_matrix[categories[test_df['class'][i]] - 1][categories[pred_class['class']
                                                                                 [matches[matches[:, 0] == i].tolist()[0][1]]] - 1] += 1
            else:
                confusion_matrix[categories[test_df['class']
                                            [i]] - 1][confusion_matrix.shape[1] - 1] += 1

        for i in range(len(detection_boxes)):
            if matches.shape[0] > 0 and matches[matches[:, 1] == i].shape[0] == 0:
                confusion_matrix[confusion_matrix.shape[0] -
                                 1][categories[pred_class['class'][i]] - 1] += 1

    return confusion_matrix


def create_CM(ground_truth_csv, pred_csv, IOU_THRESHOLD=0.3, CONFIDENCE_THRESHOLD=0.3):
    precision = 0.0
    recall = 0.0
    ground_truth_csv['filename'] = ground_truth_csv['filename'].apply(
        lambda x: x.split('\\')[-1])

    pred_csv['filename'] = pred_csv['filename'].apply(
        lambda x: x.split('\\')[-1])
    label_map = "C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\labels_dict.json"
    output_path = "confusion_matrix.csv"
    with open(label_map, "r") as json_file:
        categories = json.load(json_file)

    results = []
    class_uniq = pred_csv['class'].unique()
    confusion_matrix = process_detections(
        ground_truth_csv, pred_csv, IOU_THRESHOLD, CONFIDENCE_THRESHOLD, categories)
    for label in class_uniq:
        class_id = int(float(categories[label]))-1
        name = label

        total_target = np.sum(confusion_matrix[class_id, :])
        total_predicted = np.sum(confusion_matrix[:, class_id])

        # precision = float(
        #     confusion_matrix[class_id, class_id] / total_predicted)
        # recall = float(confusion_matrix[class_id, class_id] / total_target)
        try:
            precision = float(confusion_matrix[class_id, class_id] / total_predicted)
            if math.isnan(precision):
                precision = 0.0
        except Exception:
            precision = 0.0

        try:
            recall = float(confusion_matrix[class_id, class_id] / total_target)
            if math.isnan(recall):
                recall = 0.0
        except Exception:
            recall = 0.0

        results.append({'category': name, 'precision_@{}IOU'.format(IOU_THRESHOLD): precision, 'recall_@{}IOU'.format(IOU_THRESHOLD): recall})
    print(confusion_matrix)
    df = pd.DataFrame(results)
    print(df)
    df.to_csv(output_path)

In [ ]:
create_CM(ground_truth_csv, pred_data_read,IOU_THRESHOLD=0.1,CONFIDENCE_THRESHOLD=0.1)

In [ ]:
%cd C:\\Users\\Imtiaz\\Downloads\\DL_models\\utils

In [ ]:
import time
import subprocess

start_time = time.time()

command = ['python', 'confusion_matrix_tf2.py',
           '--input_tfrecord_path', 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\test_new.record',
           '--output_path', 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\metrics',
           '--inference_graph', 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\saved_models\\newFRCNNRESNET50EXP1\\saved_model',
           '--class_labels', 'C:\\Users\\Imtiaz\\Downloads\\DL_models\\annotations\\labelmap.pbtxt']

result = subprocess.run(command, capture_output=True, text=True)

end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")
print("Output:", result.stdout)


In [ ]:
!python confusion_matrix_tf2.py --input_tfrecord_path xx --output_path xx --inference_graph xx --class_labels xx